In [1]:
import polars as pl

train_data = pl.read_csv("./train.csv", has_header=True)
test_data = pl.read_csv("./test.csv", has_header=True)
train_data.head()

FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,tif_path
i64,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,i64,i64,i64,str,str,str
1326576,"""Healthy""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-11-25 0:00:00""","""2024-04-14 0:00:00""",97,54,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Groundwater""",4,87,17,"""Rabi""","""POLYGON ((78.18079255482755 17…","""downloads/20240412T050649_2024…"
1326577,"""Healthy""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-11-13 0:00:00""","""2024-04-26 0:00:00""",82,58,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,94,15,"""Rabi""","""POLYGON ((78.17483419891283 17…","""downloads/20240425T051649_2024…"
1326578,"""Healthy""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-12-19 0:00:00""","""2024-04-28 0:00:00""",92,91,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",3,99,20,"""Rabi""","""POLYGON ((78.16888476438905 17…","""downloads/20240427T050651_2024…"
1331836,"""Diseased""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-02-11 0:00:00""","""2024-11-04 0:00:00""",91,52,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,92,16,"""Rabi""","""POLYGON ((78.16821795882959 17…","""downloads/20241103T050951_2024…"
1326579,"""Diseased""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-12-12 0:00:00""","""2024-05-19 0:00:00""",94,55,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,97,20,"""Rabi""","""POLYGON ((78.17244459940454 17…","""downloads/20240517T050651_2024…"


In [2]:
train_data = train_data.drop("State")
test_data = test_data.drop("State")
train_data = train_data.with_columns(pl.col("category").alias("Target")).drop("category")
test_data = test_data.with_columns(pl.col("category").alias("Target")).drop("category")

In [3]:
train_data = train_data.with_columns(pl.col("SDate").str.to_date("%Y-%m-%d %H:%M:%S").alias("SDate"))
train_data = train_data.with_columns(pl.col("HDate").str.to_date("%Y-%m-%d %H:%M:%S").alias("HDate"))

test_data = test_data.with_columns(pl.col("SDate").str.to_date("%Y-%m-%d %H:%M:%S").alias("SDate"))
test_data = test_data.with_columns(pl.col("HDate").str.to_date("%Y-%m-%d %H:%M:%S").alias("HDate"))
train_data.drop("FarmID", "Crop", "District", "Sub-District").head()

SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,tif_path,Target
date,date,i64,i64,str,str,str,str,str,i64,i64,i64,str,str,str,str
2023-11-25,2024-04-14,97,54,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Groundwater""",4,87,17,"""Rabi""","""POLYGON ((78.18079255482755 17…","""downloads/20240412T050649_2024…","""Healthy"""
2023-11-13,2024-04-26,82,58,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,94,15,"""Rabi""","""POLYGON ((78.17483419891283 17…","""downloads/20240425T051649_2024…","""Healthy"""
2023-12-19,2024-04-28,92,91,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",3,99,20,"""Rabi""","""POLYGON ((78.16888476438905 17…","""downloads/20240427T050651_2024…","""Healthy"""
2023-02-11,2024-11-04,91,52,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,92,16,"""Rabi""","""POLYGON ((78.16821795882959 17…","""downloads/20241103T050951_2024…","""Diseased"""
2023-12-12,2024-05-19,94,55,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,97,20,"""Rabi""","""POLYGON ((78.17244459940454 17…","""downloads/20240517T050651_2024…","""Diseased"""


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from skimage import exposure

# Function to scale pixel values to the 0–255 range for better visualization
def scale_band(band):
    """
    Scales pixel values of a single band to the 0-255 range.

    Parameters:
    - band: np.array, pixel values of the band

    Returns:
    - np.array, scaled pixel values in the 0-255 range
    """
    band = band.astype(np.float32)  # Ensure values are in float for scaling
    return 255 * (band - np.min(band)) / (np.max(band) - np.min(band))  # Scale to 0–255

# Function to visualize RGB image with scaling, contrast adjustments, and optional gamma correction
def visualize_rgb_image(file_path, gamma=0.6, contrast_stretch=True):
    """
    Visualize an RGB image using matplotlib with scaling, optional gamma correction, and contrast stretching.

    Parameters:
    - file_path: str, path to the RGB image file (GeoTIFF)
    - gamma: float, gamma correction factor (default=0.6)
    - contrast_stretch: bool, whether to apply contrast stretching (default=True)
    """
    # Open the image file
    with rasterio.open(file_path) as src:
        # Read RGB bands (assuming Sentinel-2 band order: Red=B4, Green=B3, Blue=B2)
        red = src.read(3)  # Band 4 for Red
        green = src.read(2)  # Band 3 for Green
        blue = src.read(1)  # Band 2 for Blue

        # Scale each band to the 0–255 range for better visualization
        red_scaled = scale_band(red)
        green_scaled = scale_band(green)
        blue_scaled = scale_band(blue)

        # Stack the scaled RGB bands into a single image
        rgb = np.dstack((red_scaled, green_scaled, blue_scaled)).astype(np.uint8)

        # Apply contrast stretching if specified
        if contrast_stretch:
            p2, p98 = np.percentile(rgb, (2, 98))  # Calculate 2nd and 98th percentiles for stretching
            rgb = exposure.rescale_intensity(rgb, in_range=(p2, p98))

        # Apply gamma correction to adjust brightness
        rgb = exposure.adjust_gamma(rgb, gamma=gamma)

        # Display the processed image using matplotlib
        plt.figure(figsize=(5, 5))
        plt.imshow(rgb)
        plt.axis('off')  # Hide axes for a cleaner look
        plt.title("RGB Composite (Red-Green-Blue) with Scaling, Contrast Stretch, and Gamma Correction")
        plt.show()

    return rgb

In [5]:
# Example feature calculation functions
def calculate_ndvi(nir_band, red_band):
    """Calculate NDVI (Normalized Difference Vegetation Index)."""
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return np.nanmean(ndvi)

def calculate_evi(nir_band, red_band, blue_band):
    """Calculate EVI (Enhanced Vegetation Index)."""
    evi = 2.5 * (nir_band - red_band) / (nir_band + 6 * red_band - 7.5 * blue_band + 1)
    return np.nanmean(evi)

def calculate_ndwi(nir_band, green_band):
    """Calculate NDWI (Normalized Difference Water Index)."""
    ndwi = (green_band - nir_band) / (green_band + nir_band)
    return np.nanmean(ndwi)

def calculate_gndvi(nir_band, green_band):
    """Calculate GNDVI (Green Normalized Difference Vegetation Index)."""
    gndvi = (nir_band - green_band) / (nir_band + green_band)
    return np.nanmean(gndvi)

def calculate_savi(nir_band, red_band, L=0.5):
    """Calculate SAVI (Soil Adjusted Vegetation Index)."""
    savi = ((nir_band - red_band) / (nir_band + red_band + L)) * (1 + L)
    return np.nanmean(savi)

def calculate_msavi(nir_band, red_band):
    """Calculate MSAVI (Modified Soil Adjusted Vegetation Index)."""
    msavi = (2 * nir_band + 1 - np.sqrt((2 * nir_band + 1)**2 - 8 * (nir_band - red_band))) / 2
    return np.nanmean(msavi)

In [6]:
import os
import sys
sys.path.append('../')
from my_configs.configs import Config

# Function to calculate features for each row in the data
def process_row_for_features(index, row):
    features = {'index': index}
    print(f"Processing row {index}...")

    # Retrieve and validate the TIFF file path
    tif_path = row.get('tif_path')
    if not isinstance(tif_path, str) or not os.path.exists(os.path.join(Config.ROOT_PATH, tif_path)):
        print(f"Skipping entry due to missing or invalid tif_path for index {index}")
        features.update({
            'ndvi': np.nan, 'evi': np.nan, 'ndwi': np.nan,
            'gndvi': np.nan, 'savi': np.nan, 'msavi': np.nan
        })
        return features

    # Rest of the processing remains unchanged
    try:
        with rasterio.open(os.path.join(Config.ROOT_PATH, tif_path)) as src:
            red = src.read(3)
            green = src.read(2)
            blue = src.read(1)
            nir = src.read(4)

            features['ndvi'] = calculate_ndvi(nir, red)
            features['evi'] = calculate_evi(nir, red, blue)
            features['ndwi'] = calculate_ndwi(nir, green)
            features['gndvi'] = calculate_gndvi(nir, green)
            features['savi'] = calculate_savi(nir, red)
            features['msavi'] = calculate_msavi(nir, red)
    except Exception as e:
        print(f"Error processing {tif_path}: {e}")
        features.update({'ndvi': np.nan, 'evi': np.nan, 'ndwi': np.nan,
                         'gndvi': np.nan, 'savi': np.nan, 'msavi': np.nan})
    return features

# Use parallel processing to calculate features for each row
# `tqdm` provides a progress bar to track completion
from joblib import Parallel, delayed
from tqdm import tqdm

train_new_features = Parallel(n_jobs=-1)(delayed(process_row_for_features)(index, row)
                                   for index, row in tqdm(train_data.to_pandas().iterrows(), total=len(train_data)))
test_new_features = Parallel(n_jobs=-1)(delayed(process_row_for_features)(index, row)
                                    for index, row in tqdm(test_data.to_pandas().iterrows(), total=len(test_data)))

100%|██████████| 2718/2718 [00:06<00:00, 424.93it/s]


In [7]:
import pandas as pd
train_new_features_df = pd.DataFrame(train_new_features).to_csv("train_new_features.csv", index=False)
test_new_features_df = pd.DataFrame(test_new_features).to_csv("test_new_features.csv",index=False)

In [8]:
import pandas as pd
train_new_features_df = pl.read_csv("train_new_features.csv").drop("index")
test_new_features_df = pl.read_csv("test_new_features.csv").drop("index")
train_new_features_df.describe()

statistic,ndvi,evi,ndwi,gndvi,savi,msavi
str,f64,f64,f64,f64,f64,f64
"""count""",7463.0,7472.0,7463.0,7463.0,7472.0,7472.0
"""null_count""",425.0,416.0,425.0,425.0,416.0,416.0
"""mean""",0.691944,inf,10.231093,0.290997,1.027334,3837.78398
"""std""",1.674369,NaN,2.775101,0.882227,2.498257,1512.270207
"""min""",0.004725,-1479.291647,0.0066,0.003807,0.0,0.0
"""25%""",0.122154,-0.964962,9.441717,0.101655,0.180797,3039.070068
"""50%""",0.18219,0.708537,10.930205,0.167471,0.271772,3451.486328
"""75%""",0.262071,3.078382,11.976568,0.214791,0.391348,3964.207275
"""max""",18.339667,inf,16.823843,15.752436,27.505635,12733.712891


In [9]:
train_data = train_data.hstack(train_new_features_df)
test_data = test_data.hstack(test_new_features_df)
train_data.null_count()

FarmID,Crop,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,tif_path,Target,ndvi,evi,ndwi,gndvi,savi,msavi
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,425,416,425,425,416,416


In [10]:
train_data = train_data.drop("SDate", "HDate", "geometry", 'evi', "tif_path").fill_null(strategy="min")
test_data = test_data.drop("SDate", "HDate", "geometry", 'evi', "tif_path").fill_null(strategy="min")
train_data.null_count()

FarmID,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,Target,ndvi,ndwi,gndvi,savi,msavi
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
test_data.null_count()

FarmID,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,Target,ndvi,ndwi,gndvi,savi,msavi
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2718,0,0,0,0,0


In [12]:
X = train_data.drop("FarmID", 'Target').to_pandas()
y = train_data.select("Target").to_pandas()

In [16]:
categorical_features

Index(['Crop', 'District', 'Sub-District', 'CNext', 'CLast', 'CTransp',
       'IrriType', 'IrriSource', 'Season'],
      dtype='object')

In [36]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import numpy as np

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0, stratify=y)


# Define features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define preprocessing for numeric features (impute missing values and scale them)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', 'passthrough', categorical_features)])

# class weights
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train.values.ravel())
print(class_weights)
# Define the model
cat_features_indices = list(range(len(numeric_features), len(numeric_features) + len(categorical_features)))
model = CatBoostClassifier(iterations=1000, verbose=100, cat_features=cat_features_indices, class_weights=class_weights)

# Create the pipeline
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', model)
                      ])
pipe

# Fit the model
pipe.fit(X_train, y_train)

# Then evaluate using the pipeline on the validation set
print("Validation accuracy:", pipe.score(X_valid, y_valid))

[4.16226913 0.30307397 4.06572165 4.66715976]
Learning rate set to 0.086921
0:	learn: 1.3794118	total: 64.8ms	remaining: 1m 4s
100:	learn: 1.1263394	total: 6.06s	remaining: 53.9s
200:	learn: 0.8700242	total: 12.3s	remaining: 48.7s
300:	learn: 0.6959878	total: 18.6s	remaining: 43.2s
400:	learn: 0.5673204	total: 25.2s	remaining: 37.7s
500:	learn: 0.4730720	total: 31.9s	remaining: 31.7s
600:	learn: 0.4014668	total: 38.3s	remaining: 25.4s
700:	learn: 0.3467962	total: 45.3s	remaining: 19.3s
800:	learn: 0.3003041	total: 53.1s	remaining: 13.2s
900:	learn: 0.2637295	total: 1m	remaining: 6.59s
999:	learn: 0.2332646	total: 1m 6s	remaining: 0us
Validation accuracy: 0.7186311787072244


In [37]:
y_pred = pipe.predict(X_valid)
from sklearn.metrics import classification_report
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

    Diseased       0.14      0.16      0.15        95
     Healthy       0.83      0.85      0.84      1301
       Pests       0.11      0.09      0.10        97
    Stressed       0.06      0.04      0.04        85

    accuracy                           0.72      1578
   macro avg       0.28      0.28      0.28      1578
weighted avg       0.70      0.72      0.71      1578



In [38]:
# Load submission file
import polars as pl
sample_submission = pl.read_csv( '../telangana_data/data/SampleSubmission.csv')

# Predict on the test set
# test_encoded['target'] = model.predict(test_encoded.drop(columns=['FarmID', 'dataset', 'target']))  # Drop 'FarmID' if it exists in test_encoded
# test_encoded['target'] = test_encoded['target'].astype(int).map(idx_to_category_mapper)
# test_encoded_pl = pl.from_pandas(test_encoded)

# sample_submission = sample_submission.join(test_encoded_pl, left_on='ID', right_on='FarmID').select(['ID', 'target'])
X_sub = test_data.drop("FarmID", 'Target').to_pandas()

X_valid_processed = preprocessor.transform(X_sub)

y_sub = model.predict(X_valid_processed)

In [39]:
submission = test_data.with_columns(pl.Series(name='Target',values=y_sub.reshape(-1))).select("FarmID", "Target")
submission.write_csv('submission.csv')
submission

FarmID,Target
i64,str
85197,"""Healthy"""
779677,"""Healthy"""
1331840,"""Healthy"""
483053,"""Healthy"""
542945,"""Healthy"""
…,…
551119,"""Healthy"""
916476,"""Healthy"""
121860,"""Healthy"""


In [40]:
submission['Target'].value_counts()

Target,count
str,u32
"""Stressed""",6
"""Pests""",11
"""Diseased""",174
"""Healthy""",2527
